In [1]:
from pyspark.sql.types import *
import pyspark.sql.functions  as f
from pyspark.sql.functions import explode
import time


In [2]:
df = spark.createDataFrame([
    ('id_cliente-1',  'cat-1, cat-2, cat-3'),
    ('id_cliente-2',  'cat-1, cat-4, cat-5'),
    ('id_cliente-3',  'cat-6, cat-7'),
    ('id_cliente-4',  'cat-1, cat-2, cat-7, cat-10'),
    ('id_cliente-5',  'cat-8, cat-10'),
    ('id_cliente-6',  'cat-1, cat-9, cat-10'),
    ('id_cliente-7',  'cat-1, cat-4, cat-5, cat-10'),
    ('id_cliente-8',  'cat-7, cat-9'),
    ('id_cliente-9',  'cat-1'),
    ('id_cliente-10', 'cat-1, cat-2, cat-3, cat-4, cat-5, cat-6, cat-7, cat-8, cat-10')
], ['id_cliente', 'categorias'])

In [3]:
df2 = spark.createDataFrame([
    ('id_cliente-1',  'cat-1, cat-2, cat-3, cat-15'),
    ('id_cliente-2',  'cat-1, cat-4, cat-5, cat-11, cat-14'),
    ('id_cliente-3',  'cat-4, cat-14, cat-15'),
    ('id_cliente-4',  'cat-1, cat-2, cat-7, cat-10'),
    ('id_cliente-5',  'cat-8, cat-10, cat-11'),
    ('id_cliente-6',  'cat-1, cat-9, cat-10, cat-11, cat-13'),
    ('id_cliente-7',  'cat-1, cat-4, cat-5, cat-10'),
    ('id_cliente-8',  'cat-7, cat-9, cat-12, cat-13, cat-14'),
    ('id_cliente-9',  'cat-2'),
    ('id_cliente-10', 'cat-1, cat-2, cat-3, cat-4, cat-5, cat-6, cat-7, cat-8, cat-10')
], ['id_cliente', 'categorias'])

In [4]:
start = time.time()

dfx = df2.select(
        "id_cliente",
        f.split("categorias", ", ").alias("categorias"),
        f.posexplode(f.split("categorias", ", ")).alias("pos", "val"), f.lit("1").alias("categ")
    )\
    .groupBy("id_cliente").pivot("val").agg(f.first("categ"))\
    .orderBy(f.length("id_cliente"),"id_cliente" )\
    .fillna("0")

pivotColumns = dfx.columns
c0 = pivotColumns.pop(0)
def order(item):
  return int(item.split('-')[1])
pivotColumns = sorted(pivotColumns, key=order)
pivotColumns.insert(0,c0)

dfx.select(pivotColumns).show()


print(time.time() - start)

+-------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+
 id_cliente|cat-1|cat-2|cat-3|cat-4|cat-5|cat-6|cat-7|cat-8|cat-9|cat-10|cat-11|cat-12|cat-13|cat-14|cat-15|
+-------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+
 id_cliente-1| 1| 1| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1|
 id_cliente-2| 1| 0| 0| 1| 1| 0| 0| 0| 0| 0| 1| 0| 0| 1| 0|
 id_cliente-3| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 1|
 id_cliente-4| 1| 1| 0| 0| 0| 0| 1| 0| 0| 1| 0| 0| 0| 0| 0|
 id_cliente-5| 0| 0| 0| 0| 0| 0| 0| 1| 0| 1| 1| 0| 0| 0| 0|
 id_cliente-6| 1| 0| 0| 0| 0| 0| 0| 0| 1| 1| 1| 0| 1| 0| 0|
 id_cliente-7| 1| 0| 0| 1| 1| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0|
 id_cliente-8| 0| 0| 0| 0| 0| 0| 1| 0| 1| 0| 0| 1| 1| 1| 0|
 id_cliente-9| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
id_cliente-10| 1| 1| 1| 1| 1| 1| 1| 1| 0| 1| 0| 0| 0| 0| 0|
+-------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+

1.7952461242675781